<a href="https://colab.research.google.com/github/hlecuanda/jupyter-notebooks-of-all-kinds/blob/master/Python_Requests_Rapidamente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción rápida al uso de la librería `requests`

Aquí veremos de manera concisa la forma correcta de invocar un request simple a un api HTTP, limpiando nuestras conecciones, y validando haber recibido una respuesta correcta del servidor antes de procesarla. 

## setup basico. 

Importamos algunas librerias y la linida impresora para ver nuestros objetos

In [48]:
import requests
import json
import pprint


# una linda impresora con indentacion, maxima profundiad 3 
# no-compacta
pp = pprint.PrettyPrinter(indent=2,depth=3,compact=False)


<Response [200]>


## El ejemplo mas básico de uso de la libreria requests

Hacemos un `get request` a un api que solo responde con tu propio IP en un objeto json simple (el codigo del servidor lo puedes ver al final)

In [90]:

# un api estupido que responde con tu propio IP en un mensaje json
url = 'https://mezaops.appspot.com/knock/'
r = requests.get(url)

pp.pprint(r)

<Response [200]>


In [49]:
pp.pprint(r.content) 

b'{\n    "ip": "35.225.45.29"\n}\n'


aqui `r.content` es una propiedad del objeto `r` definido por `requests` y contiene un `bytearray` con la respuesta del servidor, que en este caso es un objeto json, con una sola propiedad, "ip" cuyo valor es tu propio IP

el `bytearray` lo debemos convertir a un objeto python para poder usarlo.  requests usa `bytearray` para el contenido de la respuesta porque es el tipo mas generico en el que cabe cualquier respuesa, ya sea texto puro, o bits brutos binarios de una imagen. solo tu sabes que es lo que esperas de el servidor. 

para poder usarlo, lo pasamos por `json.loads` que automaticamente detecta el tipo texto y el encoding `utf8` que es el default para el tipo mime `application/json`

In [50]:
# eso es json en  bruto, para poderlo usar, pasalo por json.loads() 
# loads quiere decir "load string"

response_content_object = json.loads(r.content)

print('el objeto diccionario completo:')
pp.pprint(response_content_object) # response_obect es un diccionario con un solo elemento: 'ip'.

print('\n')
print('el elemento "ip" en el diccionario:')
pp.pprint(response_content_object['ip'])


el objeto diccionario completo:
{'ip': '35.225.45.29'}


el elemento "ip" en el diccionario:
'35.225.45.29'


## Otras propiedades de los objetos `request`

El objeto respuesta `r` contiene otras propiedades que puedes usar en el codigo para validad si has recibido una respuesta valida del servidor, notablemente el codigo de respuesta en los encabezados:

In [51]:
pp.pprint(r.status_code)

200


Una lista de todas las propiedades contenidas en el objeto `r`

In [52]:
properties = [ prop for prop in dir(r) if '_' not in prop ]

pp.pprint(properties)

[ 'close',
  'connection',
  'content',
  'cookies',
  'elapsed',
  'encoding',
  'headers',
  'history',
  'json',
  'links',
  'next',
  'ok',
  'raw',
  'reason',
  'request',
  'text',
  'url']


Veamos que es lo que hay en todo el objeto 'r' que nos pueda servir para validar la respuesta antes de procesarla, evitando un error o excepcion en caso de no recibir respuesta adecuada


In [78]:
for p in properties:
    print('Property:>>{}<<'.format(p))
    print('With Value:{}'.format(r.__getattribute__(p)))
    print('-----------------------\n')

Property:>>close<<
With Value:<bound method Response.close of <Response [200]>>
-----------------------

Property:>>connection<<
With Value:<requests.adapters.HTTPAdapter object at 0x7f20537aa9e8>
-----------------------

Property:>>content<<
With Value:b'{\n    "ip": "35.225.45.29"\n}\n'
-----------------------

Property:>>cookies<<
With Value:<RequestsCookieJar[]>
-----------------------

Property:>>elapsed<<
With Value:0:00:00.035224
-----------------------

Property:>>encoding<<
With Value:None
-----------------------

Property:>>headers<<
With Value:{'Content-Type': 'application/json', 'X-Cloud-Trace-Context': 'f673bb912acce55214567f4485fdda2e;o=1', 'Date': 'Tue, 20 Oct 2020 20:20:40 GMT', 'Server': 'Google Frontend', 'Content-Length': '29'}
-----------------------

Property:>>history<<
With Value:[]
-----------------------

Property:>>json<<
With Value:<bound method Response.json of <Response [200]>>
-----------------------

Property:>>links<<
With Value:{}
----------------------

podemos ver que `requests` amablemente provee un objeto `json` ya procesado que podemos usar directamente.

tambien nos provee un objeto `ok` con valor `True` que podemos usar para determinar si el request fue exitoso con un bonito if asi:

```
if r.ok:
     # hacemos algo c on r
else:
     # procesams el error
```

tenemos tambien `r.reason` entonces, muy utilmente podemos decir
```
if r.ok:
     # hacemos algo c on r
else:
     print('request fallo porque {}'.format(r.reason))
```

nota tambien que tenemos un metodo `r.close()` lo que nos dice que el request deja abierta una coneccion http al servidor que podemos reusar, y que debemos cerrar al terminar, 

y que tenemos un metodo `r.next()` que hace de nuestro objeto `r`` lindo iterable!! omg, que util! 

podriamos usar algo asi:

```
for x in r:
    print(x.content)
```

esto es para inspecionar conecciones redireccionadas (algunos apis aprovechan ese tipo de conecciones para generar listas de resultaos)



## Codigo generico para un request simple, con verificacion de status y manejo de errores en la coneccion

Podemos ver que el objeto respuesta que nos ofrece la libreria `requests` contiene una multitud de objetos utiles para la validacion de una respuesta del servidor. 

Hay que notar tambien, dada la presencia de un metodo `r.close()` es que la conección esta abierta, y debemos cerrarla cuando ya no sea necesaria. (es el caso análogo a cerrar un archivo despues de escribir en el) para esto usamos un administrador de contextos, de la misma manera que se hace al abrir y cerrar archivos.

Finalmente, nuestro código lo podemos escribir de la siguiente manera:



In [88]:
from contextlib import closing # contextos para cerrar conecciones http

url = 'https://mezaops.appspot.com/knock/'

with closing(requests.get(url)) as newreq: # al terminar de usar newreq, la coneccion se cerrara
    if newreq.ok:                          # tenemos una respuesta valida
        print('Request exitoso')           # se lo decimos al usuario
        print('Respuesta:')
        pp.pprint(newreq.json())           # ya tenemos un objeto json procesado 
        print('http status code: {}'.format(newreq.status_code))

    else:                                  # Algo sucedio mal. probablemente en el servidor
        print('Request a {} fallo por esta razon; {}'.format(
            newreq.url,newreq.reason))      # le avisamos al usuario que ocurrio mal
        print('http status code: {}'.format(newreq.status_code))



Request exitoso
Respuesta:
{'ip': '35.225.45.29'}
http status code: 200


El servidor responde con error 404 cuando pides un url que no es `/knock/`, veamos como se comporta: 

In [86]:
url = 'https://mezaops.appspot.com/knick/' # "knick", no "knock"

with closing(requests.get(url)) as newreq: # al terminar de usar newreq, la coneccion se cerrara
    if newreq.ok:                          # tenemos una respuesta valida
        print('Request exitoso')           # se lo decimos al usuario
        print('Respuesta:')
        pp.pprint(newreq.json())           # ya tenemos un objeto json procesado 
        print('http status code: {}'.format(newreq.status_code))

    else:                                  # Algo sucedio mal. probablemente en el servidor
        print('Request a {} fallo por esta razon; "{}"'.format(
            newreq.url,newreq.reason))      # le avisamos al usuario que ocurrio mal
        print('http status code: {}'.format(newreq.status_code))

Request a https://mezaops.appspot.com/knick/ fallo por esta razon; "Not Found"
http status code: 404


# Código fuente del servidor


El siguiente es el codigo corriendo en el servidor (Google app engine con python 2.7, usando Flask) (lleva como 4 años ahi jajaj ya tengo que actualizarlo porque py2 ya no fifa)


```python

"""  Knocker - knock to trigger an open firewall port

    %his module implements a simple service infroming the
    caller of his public IP address as seen from the cloud

"""
from flask import Flask
from werkzeug.wrappers import Response

app = Flask(__name__)

from flask import request

@app.route('/knock/', methods=['GET', 'POST'])
def whatismyip():
    """Return the callers IP plain and simple"""
    return Response('Data: %s' % request.remote.addr)

@app.errorhandler(404)
def page_not_found(e):
    """Return a custom 404 error."""
    return 'Sorry, nothing at this URL.', 404

```

-Hector Lecuanda (hector@lecuanda.com)